# Puzzle 1
--- Day 8: Playground ---

Equipped with a new understanding of teleporter maintenance, you confidently step onto the repaired teleporter pad.

You rematerialize on an unfamiliar teleporter pad and find yourself in a vast underground space which contains a giant playground!

Across the playground, a group of Elves are working on setting up an ambitious Christmas decoration project. Through careful rigging, they have suspended a large number of small electrical junction boxes.

Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes.

The Elves are trying to figure out which junction boxes to connect so that electricity can reach every junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input).

For example:

```
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
```

This list describes the position of 20 junction boxes, one per line. Each position is given as X,Y,Z coordinates. So, the first junction box in the list is at X=162, Y=817, Z=812.

To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are as close together as possible according to straight-line distance. In this example, the two junction boxes which are closest together are 162,817,812 and 425,690,689.

By connecting these two junction boxes together, because electricity can flow between them, they become part of the same circuit. After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits.

Now, the two junction boxes which are closest together but aren't already directly connected are 162,817,812 and 431,825,988. After connecting them, since 162,817,812 is already connected to another junction box, there is now a single circuit which contains three junction boxes and an additional 17 circuits which contain one junction box each.

The next two junction boxes to connect are 906,360,560 and 805,96,715. After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each.

The next two junction boxes are 431,825,988 and 425,690,689. Because these two junction boxes were already in the same circuit, nothing happens!

This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be.

After making the ten shortest connections, there are 11 circuits: one circuit which contains 5 junction boxes, one circuit which contains 4 junction boxes, two circuits which contain 2 junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces 40.

Your list contains many junction boxes; connect together the 1000 pairs of junction boxes which are closest together. Afterward, what do you get if you multiply together the sizes of the three largest circuits?


In [59]:
import numpy as np
test_input = '''162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689'''
test_boxes = [np.array(list(map(int,box.split(',')))) for box in test_input.split('\n')]
distances = np.array([[np.linalg.norm(test_boxes[i] - test_boxes[j]) if j>i else 0 for j in range(len(test_boxes))] for i in range(len(test_boxes))]) 

In [60]:
#modify for puzzle 2

class JunctionBoxes:
    boxes: list[list[int]]
    distances: np.array
    circuits: list[list[int]]

    def __init__(self, boxes, circuits = None):
        self.boxes = boxes
        self.distances = np.array([[np.linalg.norm(boxes[i] - boxes[j])
                                     if j>i else 0 for j in range(len(boxes))] 
                                     for i in range(len(boxes))]) 
        if circuits is None:
            self.circuits = [[box_num] for box_num in range(len(boxes))]
        else:
            self.circuits = circuits
    
    def _nonzero_argmin(self, arr: np.array) -> tuple[int,int]:
        """Find the index of the minimum value in the nonzero entries of an np array."""
        rows, cols = np.nonzero(arr)
        min_idx = np.argmin(arr[rows,cols])
        return rows[min_idx], cols[min_idx]
    
    def _connect_boxes(self, arr: np.array, first_box_num: int, second_box_num: int) -> np.array:
        """Connect two boxes and zero out their distances in the provided array."""
        for circuit in self.circuits:
            if first_box_num in circuit:
                first_box_circuit = circuit
            if second_box_num in circuit:
                second_box_circuit = circuit
        if first_box_circuit != second_box_circuit:
            self.circuits.remove(first_box_circuit)
            self.circuits.remove(second_box_circuit)
            self.circuits.append(first_box_circuit + second_box_circuit)
        #if they are equal, no action is needed

        arr[first_box_num, second_box_num] = 0
        arr[second_box_num, first_box_num] = 0

        return arr
    
    def connect_circuits(self, n_connections: int = 1000) -> tuple[int,int]:
        """Make the first n_connections closest junction box connections.
        Return the coordinates of the last connection made (for puzzle 2)."""
        box_array = self.distances
        for conn in range(n_connections):
            nearest_row, nearest_col = self._nonzero_argmin(box_array)
            box_array = self._connect_boxes(box_array, nearest_row, nearest_col)
        return nearest_row, nearest_col
    
    def n_largest_circuits(self, n_largest: int = 3, n_connections: int = 1000) -> int:
        self.connect_circuits(n_connections)
        self.circuits.sort(key=len, reverse = True)
        return np.prod([len(circuit) for circuit in self.circuits[:n_largest]])    
    
    def connect_until_one(self) -> int:
        """Added for puzzle 2."""
        while len(self.circuits) > 1:
            row, col = self.connect_circuits(n_connections = 1)
        return self.boxes[row][0] * self.boxes[col][0]


In [61]:
testArray = JunctionBoxes(test_boxes)
testArray.n_largest_circuits(n_largest = 3, n_connections = 10)
# 40
# success!

40

In [62]:
with open('../data/day8.txt', 'r') as input_fle:
    input_boxes = [np.array(list(map(int,box.split(',')))) for box in input_fle.readlines()]

In [63]:
junctionArray = JunctionBoxes(input_boxes)
junctionArray.n_largest_circuits(n_largest = 3, n_connections = 1000)
# 52668
# success!

52668

# Puzzle 2
--- Part Two ---

The Elves were right; they definitely don't have enough extension cables. You'll need to keep connecting junction boxes together until they're all in one large circuit.

Continuing the above example, the first connection which causes all of the junction boxes to form a single circuit is between the junction boxes at 216,146,977 and 117,168,530. The Elves need to know how far those junction boxes are from the wall so they can pick the right extension cable; multiplying the X coordinates of those two junction boxes (216 and 117) produces 25272.

Continue connecting the closest unconnected pairs of junction boxes together until they're all in the same circuit. What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?


In [64]:
testArray = JunctionBoxes(test_boxes)
testArray.connect_until_one()
# 25272
# success!

25272

In [ ]:
junctionArray = JunctionBoxes(input_boxes)
junctionArray.connect_until_one()
# 1474050600
# success!

1474050600

# Reflection

This answer is inefficient; I should just find the first n pairs which will be merged then do the merging in one step. But it wasn't inefficient *enough* to stop me from solving puzzle 2 today: this solution ran on the input in about 90 seconds.